In [ ]:
#!/usr/bin/env python3
"""
EcoGuard AI - Real-Time Environmental Monitoring and Pollution Intervention System
Complete implementation for Google Colab with Gradio deployment
"""

# Install required packages (run this cell first in Colab)
import subprocess
import sys

def install_packages():
    packages = [
        'gradio',
        'plotly',
        'pandas',
        'numpy',
        'scikit-learn',
        'folium',
        'requests',
        'datetime',
        'threading',
        'queue',
        'json',
        'time',
        'random'
    ]

    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        except:
            print(f"Note: {package} may already be installed")

install_packages()

import gradio as gr
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import folium
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
import json
import time
import threading
import queue
import random
from typing import Dict, List, Tuple, Optional

class EcoGuardAI:
    """
    Main EcoGuard AI system for environmental monitoring and prediction
    """

    def __init__(self):
        self.sensor_data = queue.Queue()
        self.prediction_model = None
        self.scaler = StandardScaler()
        self.current_data = self._generate_initial_data()
        self.drone_missions = []
        self.citizen_reports = []
        self.blockchain_logs = []
        self.is_monitoring = False
        self.monitoring_thread = None

        # Initialize AI model
        self._initialize_ai_model()

        # City coordinates for deployment
        self.cities = {
            'Mumbai': {'lat': 19.0760, 'lng': 72.8777, 'pollution_base': 85},
            'Dhaka': {'lat': 23.8103, 'lng': 90.4125, 'pollution_base': 120},
            'Delhi': {'lat': 28.7041, 'lng': 77.1025, 'pollution_base': 150},
            'Beijing': {'lat': 39.9042, 'lng': 116.4074, 'pollution_base': 95},
            'Los Angeles': {'lat': 34.0522, 'lng': -118.2437, 'pollution_base': 65}
        }

    def _generate_initial_data(self) -> pd.DataFrame:
        """Generate realistic initial sensor data"""
        dates = pd.date_range(start='2024-01-01', end='2025-01-01', freq='H')

        data = []
        for i, date in enumerate(dates):
            # Simulate realistic pollution patterns
            hour = date.hour
            day_of_week = date.dayofweek

            # Base pollution levels with daily and weekly patterns
            pm25_base = 45 + 20 * np.sin(2 * np.pi * hour / 24) + 10 * np.sin(2 * np.pi * day_of_week / 7)
            co2_base = 400 + 50 * np.sin(2 * np.pi * hour / 24)

            # Add random variation
            pm25 = max(0, pm25_base + np.random.normal(0, 10))
            co2 = max(350, co2_base + np.random.normal(0, 20))

            # Water quality parameters
            ph = 7.0 + np.random.normal(0, 0.5)
            heavy_metals = max(0, 0.1 + np.random.normal(0, 0.05))

            # Weather factors
            temperature = 25 + 10 * np.sin(2 * np.pi * (i % 8760) / 8760) + np.random.normal(0, 3)
            humidity = 60 + 20 * np.sin(2 * np.pi * (i % 8760) / 8760) + np.random.normal(0, 10)
            wind_speed = max(0, 5 + np.random.normal(0, 2))

            data.append({
                'timestamp': date,
                'pm25': pm25,
                'co2': co2,
                'ph': ph,
                'heavy_metals': heavy_metals,
                'temperature': temperature,
                'humidity': humidity,
                'wind_speed': wind_speed,
                'city': 'Mumbai',  # Default city
                'sensor_id': f'SENSOR_{i % 100:03d}'
            })

        return pd.DataFrame(data)

    def _initialize_ai_model(self):
        """Initialize and train the AI prediction model"""
        # Prepare features for ML model
        features = ['pm25', 'co2', 'ph', 'heavy_metals', 'temperature', 'humidity', 'wind_speed']

        # Create lagged features for time series prediction
        for feature in features:
            self.current_data[f'{feature}_lag1'] = self.current_data[feature].shift(1)
            self.current_data[f'{feature}_lag24'] = self.current_data[feature].shift(24)

        # Add time-based features
        self.current_data['hour'] = self.current_data['timestamp'].dt.hour
        self.current_data['day_of_week'] = self.current_data['timestamp'].dt.dayofweek
        self.current_data['month'] = self.current_data['timestamp'].dt.month

        # Remove rows with NaN values
        train_data = self.current_data.dropna()

        # Prepare training data
        feature_columns = [col for col in train_data.columns if col not in ['timestamp', 'city', 'sensor_id']]
        X = train_data[feature_columns]
        y = train_data['pm25']  # Predict PM2.5 as primary target

        # Scale features
        X_scaled = self.scaler.fit_transform(X)

        # Train model
        self.prediction_model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.prediction_model.fit(X_scaled, y)

        print("AI Model initialized and trained successfully!")

    def predict_pollution(self, hours_ahead: int = 24) -> Dict:
        """Predict pollution levels for the next N hours"""
        if self.prediction_model is None:
            return {"error": "Model not initialized"}

        # Get latest data point
        latest_data = self.current_data.iloc[-1:].copy()

        predictions = []
        current_time = datetime.now()

        for i in range(hours_ahead):
            # Create prediction timestamp
            pred_time = current_time + timedelta(hours=i+1)

            # Prepare features for prediction
            pred_features = latest_data.copy()
            pred_features['hour'] = pred_time.hour
            pred_features['day_of_week'] = pred_time.weekday()
            pred_features['month'] = pred_time.month

            # Select feature columns
            feature_columns = [col for col in pred_features.columns if col not in ['timestamp', 'city', 'sensor_id']]
            X_pred = pred_features[feature_columns]

            # Scale features
            X_pred_scaled = self.scaler.transform(X_pred)

            # Make prediction
            predicted_pm25 = self.prediction_model.predict(X_pred_scaled)[0]

            predictions.append({
                'timestamp': pred_time,
                'predicted_pm25': predicted_pm25,
                'risk_level': self._get_risk_level(predicted_pm25)
            })

        return {
            'predictions': predictions,
            'alert_needed': any(pred['risk_level'] == 'High' for pred in predictions)
        }

    def _get_risk_level(self, pm25_value: float) -> str:
        """Determine risk level based on PM2.5 value"""
        if pm25_value < 35:
            return 'Low'
        elif pm25_value < 75:
            return 'Moderate'
        else:
            return 'High'

    def deploy_drone_mission(self, location: str, mission_type: str, target_coordinates: Tuple[float, float]) -> Dict:
        """Simulate drone deployment for pollution intervention"""
        mission_id = f"MISSION_{len(self.drone_missions) + 1:04d}"

        mission = {
            'mission_id': mission_id,
            'location': location,
            'type': mission_type,
            'coordinates': target_coordinates,
            'status': 'Deployed',
            'timestamp': datetime.now(),
            'estimated_completion': datetime.now() + timedelta(hours=2),
            'resources_used': {
                'drone_count': random.randint(1, 5),
                'reagent_volume': random.randint(10, 100),
                'battery_usage': random.randint(20, 80)
            }
        }

        self.drone_missions.append(mission)

        # Add to blockchain log
        self._add_blockchain_entry('drone_deployment', mission)

        return mission

    def process_citizen_report(self, location: str, pollution_type: str, severity: int, description: str) -> Dict:
        """Process citizen pollution report"""
        report_id = f"REPORT_{len(self.citizen_reports) + 1:04d}"

        report = {
            'report_id': report_id,
            'location': location,
            'pollution_type': pollution_type,
            'severity': severity,
            'description': description,
            'timestamp': datetime.now(),
            'status': 'Verified',
            'tokens_earned': severity * 10,  # Reward based on severity
            'verification_score': random.uniform(0.7, 1.0)
        }

        self.citizen_reports.append(report)

        # Add to blockchain log
        self._add_blockchain_entry('citizen_report', report)

        # If high severity, trigger drone mission
        if severity >= 4:
            self.deploy_drone_mission(location, 'emergency_response', (0, 0))

        return report

    def _add_blockchain_entry(self, entry_type: str, data: Dict):
        """Add entry to blockchain log"""
        block = {
            'block_id': len(self.blockchain_logs) + 1,
            'timestamp': datetime.now(),
            'type': entry_type,
            'data': data,
            'hash': f"hash_{random.randint(100000, 999999)}",
            'previous_hash': self.blockchain_logs[-1]['hash'] if self.blockchain_logs else 'genesis'
        }

        self.blockchain_logs.append(block)

    def generate_map(self, city: str = 'Mumbai') -> str:
        """Generate interactive map with sensor locations and pollution data"""
        city_info = self.cities.get(city, self.cities['Mumbai'])

        # Create base map
        m = folium.Map(
            location=[city_info['lat'], city_info['lng']],
            zoom_start=12,
            tiles='OpenStreetMap'
        )

        # Add sensor locations
        for i in range(20):  # 20 sensors per city
            lat = city_info['lat'] + np.random.normal(0, 0.05)
            lng = city_info['lng'] + np.random.normal(0, 0.05)

            # Generate sensor data
            pm25 = city_info['pollution_base'] + np.random.normal(0, 20)
            risk_level = self._get_risk_level(pm25)

            # Color based on risk level
            color = {'Low': 'green', 'Moderate': 'yellow', 'High': 'red'}[risk_level]

            folium.CircleMarker(
                location=[lat, lng],
                radius=8,
                popup=f"Sensor {i+1}<br>PM2.5: {pm25:.1f}<br>Risk: {risk_level}",
                color=color,
                fill=True,
                fillColor=color,
                fillOpacity=0.7
            ).add_to(m)

        # Add drone locations
        for mission in self.drone_missions[-5:]:  # Show last 5 missions
            folium.Marker(
                location=[city_info['lat'] + np.random.normal(0, 0.02),
                         city_info['lng'] + np.random.normal(0, 0.02)],
                popup=f"Drone Mission<br>ID: {mission['mission_id']}<br>Status: {mission['status']}",
                icon=folium.Icon(color='blue', icon='plane')
            ).add_to(m)

        # Save map to HTML string
        map_html = m._repr_html_()
        return map_html

    def generate_dashboard_data(self) -> Dict:
        """Generate comprehensive dashboard data"""
        current_time = datetime.now()

        # Current pollution levels
        current_pm25 = self.current_data['pm25'].iloc[-1]
        current_co2 = self.current_data['co2'].iloc[-1]
        current_ph = self.current_data['ph'].iloc[-1]

        # Predictions
        predictions = self.predict_pollution(24)

        # Statistics
        total_sensors = 500
        active_drones = len([m for m in self.drone_missions if m['status'] == 'Deployed'])
        citizen_reports_today = len([r for r in self.citizen_reports
                                   if r['timestamp'].date() == current_time.date()])

        return {
            'current_levels': {
                'pm25': current_pm25,
                'co2': current_co2,
                'ph': current_ph,
                'risk_level': self._get_risk_level(current_pm25)
            },
            'predictions': predictions,
            'statistics': {
                'total_sensors': total_sensors,
                'active_drones': active_drones,
                'citizen_reports_today': citizen_reports_today,
                'blockchain_entries': len(self.blockchain_logs)
            },
            'recent_missions': self.drone_missions[-5:],
            'recent_reports': self.citizen_reports[-10:]
        }

# Initialize the EcoGuard AI system
ecoguard = EcoGuardAI()

def create_pollution_chart():
    """Create pollution trends chart"""
    df = ecoguard.current_data.tail(168)  # Last 7 days

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('PM2.5 Levels', 'CO2 Levels', 'pH Levels', 'Heavy Metals'),
        specs=[[{'secondary_y': False}, {'secondary_y': False}],
               [{'secondary_y': False}, {'secondary_y': False}]]
    )

    # PM2.5
    fig.add_trace(
        go.Scatter(x=df['timestamp'], y=df['pm25'], name='PM2.5', line=dict(color='red')),
        row=1, col=1
    )

    # CO2
    fig.add_trace(
        go.Scatter(x=df['timestamp'], y=df['co2'], name='CO2', line=dict(color='blue')),
        row=1, col=2
    )

    # pH
    fig.add_trace(
        go.Scatter(x=df['timestamp'], y=df['ph'], name='pH', line=dict(color='green')),
        row=2, col=1
    )

    # Heavy Metals
    fig.add_trace(
        go.Scatter(x=df['timestamp'], y=df['heavy_metals'], name='Heavy Metals', line=dict(color='purple')),
        row=2, col=2
    )

    fig.update_layout(
        title='Environmental Monitoring Dashboard - Last 7 Days',
        showlegend=False,
        height=600
    )

    return fig

def create_prediction_chart():
    """Create pollution prediction chart"""
    predictions = ecoguard.predict_pollution(24)

    if 'error' in predictions:
        return go.Figure().add_annotation(text="Model not ready", x=0.5, y=0.5)

    pred_data = predictions['predictions']
    timestamps = [p['timestamp'] for p in pred_data]
    pm25_values = [p['predicted_pm25'] for p in pred_data]

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=timestamps,
        y=pm25_values,
        mode='lines+markers',
        name='Predicted PM2.5',
        line=dict(color='orange', width=3)
    ))

    # Add risk level zones
    fig.add_hline(y=35, line_dash="dash", line_color="green", annotation_text="Low Risk Threshold")
    fig.add_hline(y=75, line_dash="dash", line_color="red", annotation_text="High Risk Threshold")

    fig.update_layout(
        title='24-Hour Pollution Prediction',
        xaxis_title='Time',
        yaxis_title='PM2.5 (μg/m³)',
        height=400
    )

    return fig

def submit_citizen_report(location, pollution_type, severity, description):
    """Handle citizen report submission"""
    report = ecoguard.process_citizen_report(location, pollution_type, severity, description)

    response = f"""
    Report Submitted Successfully! 🎉

    Report ID: {report['report_id']}
    Location: {report['location']}
    Type: {report['pollution_type']}
    Severity: {report['severity']}/5
    Tokens Earned: {report['tokens_earned']} ECO
    Verification Score: {report['verification_score']:.2f}

    Thank you for helping protect our environment!
    """

    return response

def deploy_drone(location, mission_type):
    """Handle drone deployment"""
    mission = ecoguard.deploy_drone_mission(location, mission_type, (0, 0))

    response = f"""
    Drone Mission Deployed! 🚁

    Mission ID: {mission['mission_id']}
    Location: {mission['location']}
    Type: {mission['type']}
    Status: {mission['status']}
    Drones Deployed: {mission['resources_used']['drone_count']}
    Estimated Completion: {mission['estimated_completion'].strftime('%H:%M')}

    Mission logged on blockchain for transparency.
    """

    return response

def get_system_status():
    """Get current system status"""
    dashboard_data = ecoguard.generate_dashboard_data()

    status = f"""
    🌍 EcoGuard AI System Status

    Current Pollution Levels:
    • PM2.5: {dashboard_data['current_levels']['pm25']:.1f} μg/m³ ({dashboard_data['current_levels']['risk_level']} Risk)
    • CO2: {dashboard_data['current_levels']['co2']:.1f} ppm
    • pH: {dashboard_data['current_levels']['ph']:.1f}

    System Statistics:
    • Total Sensors: {dashboard_data['statistics']['total_sensors']}
    • Active Drones: {dashboard_data['statistics']['active_drones']}
    • Citizen Reports Today: {dashboard_data['statistics']['citizen_reports_today']}
    • Blockchain Entries: {dashboard_data['statistics']['blockchain_entries']}

    Prediction Alert: {"⚠️ High pollution predicted!" if dashboard_data['predictions']['alert_needed'] else "✅ No immediate alerts"}
    """

    return status

# Create Gradio interface
with gr.Blocks(theme=gr.themes.Soft(), title="EcoGuard AI") as demo:
    gr.HTML("""
    <div style="text-align: center; padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 10px; margin-bottom: 20px;">
        <h1>🌍 EcoGuard AI</h1>
        <h3>Real-Time Environmental Monitoring & Pollution Intervention System</h3>
        <p>Powered by AI • IoT Sensors • Drone Swarms • Blockchain • 5G Edge Computing</p>
    </div>
    """)

    with gr.Tabs():
        # Dashboard Tab
        with gr.TabItem("🎯 Dashboard"):
            with gr.Row():
                with gr.Column(scale=2):
                    pollution_chart = gr.Plot(value=create_pollution_chart(), label="Pollution Trends")
                    prediction_chart = gr.Plot(value=create_prediction_chart(), label="AI Predictions")

                with gr.Column(scale=1):
                    system_status = gr.Textbox(
                        value=get_system_status(),
                        label="System Status",
                        lines=15,
                        interactive=False
                    )

                    refresh_btn = gr.Button("🔄 Refresh Dashboard", variant="primary")

        # Citizen Reporting Tab
        with gr.TabItem("📱 Citizen Reports"):
            gr.Markdown("### Report Environmental Issues and Earn ECO Tokens")

            with gr.Row():
                with gr.Column():
                    report_location = gr.Textbox(label="Location", placeholder="e.g., Mumbai, Sector 15")
                    pollution_type = gr.Dropdown(
                        choices=["Air Pollution", "Water Contamination", "Noise Pollution", "Illegal Dumping"],
                        label="Pollution Type"
                    )
                    severity = gr.Slider(minimum=1, maximum=5, value=3, label="Severity (1-5)")
                    description = gr.Textbox(
                        label="Description",
                        placeholder="Describe the pollution issue...",
                        lines=3
                    )

                    submit_report_btn = gr.Button("📤 Submit Report", variant="primary")

                with gr.Column():
                    report_response = gr.Textbox(
                        label="Report Status",
                        lines=10,
                        interactive=False
                    )

        # Drone Operations Tab
        with gr.TabItem("🚁 Drone Operations"):
            gr.Markdown("### Autonomous Pollution Intervention")

            with gr.Row():
                with gr.Column():
                    drone_location = gr.Textbox(label="Deployment Location", placeholder="e.g., Mumbai Harbor")
                    mission_type = gr.Dropdown(
                        choices=["Oil Spill Cleanup", "Air Purification", "Water Treatment", "Emergency Response"],
                        label="Mission Type"
                    )

                    deploy_btn = gr.Button("🚁 Deploy Drones", variant="primary")

                with gr.Column():
                    drone_response = gr.Textbox(
                        label="Mission Status",
                        lines=10,
                        interactive=False
                    )

        # Analytics Tab
        with gr.TabItem("📊 Analytics"):
            gr.Markdown("### Environmental Data Analytics")

            city_selector = gr.Dropdown(
                choices=list(ecoguard.cities.keys()),
                value="Mumbai",
                label="Select City"
            )

            analytics_output = gr.HTML(label="City Analytics")

            def update_analytics(city):
                return f"""
                <div style="padding: 20px; background: #f8f9fa; border-radius: 10px;">
                    <h3>📍 {city} Environmental Analytics</h3>
                    <p><strong>Base Pollution Level:</strong> {ecoguard.cities[city]['pollution_base']} μg/m³</p>
                    <p><strong>Coordinates:</strong> {ecoguard.cities[city]['lat']:.4f}, {ecoguard.cities[city]['lng']:.4f}</p>
                    <p><strong>Risk Assessment:</strong> {ecoguard._get_risk_level(ecoguard.cities[city]['pollution_base'])}</p>
                    <p><strong>Recommendation:</strong> {"Immediate intervention required" if ecoguard.cities[city]['pollution_base'] > 100 else "Monitor closely"}</p>
                </div>
                """

            city_selector.change(update_analytics, inputs=[city_selector], outputs=[analytics_output])

        # Blockchain Tab
        with gr.TabItem("🔗 Blockchain Transparency"):
            gr.Markdown("### Immutable Environmental Data Logs")

            blockchain_data = gr.Dataframe(
                headers=["Block ID", "Timestamp", "Type", "Hash"],
                value=[[b['block_id'], b['timestamp'].strftime('%Y-%m-%d %H:%M'), b['type'], b['hash']]
                       for b in ecoguard.blockchain_logs[-10:]],
                label="Recent Blockchain Entries"
            )

            refresh_blockchain_btn = gr.Button("🔄 Refresh Blockchain", variant="secondary")

    # Event handlers
    refresh_btn.click(
        fn=lambda: (create_pollution_chart(), create_prediction_chart(), get_system_status()),
        outputs=[pollution_chart, prediction_chart, system_status]
    )

    submit_report_btn.click(
        fn=submit_citizen_report,
        inputs=[report_location, pollution_type, severity, description],
        outputs=[report_response]
    )

    deploy_btn.click(
        fn=deploy_drone,
        inputs=[drone_location, mission_type],
        outputs=[drone_response]
    )

    refresh_blockchain_btn.click(
        fn=lambda: [[b['block_id'], b['timestamp'].strftime('%Y-%m-%d %H:%M'), b['type'], b['hash']]
                   for b in ecoguard.blockchain_logs[-10:]],
        outputs=[blockchain_data]
    )

    # Initialize analytics
    analytics_output.value = update_analytics("Mumbai")

# Launch the application
if __name__ == "__main__":
    print("🌍 EcoGuard AI System Starting...")
    print("Features loaded:")
    print("✅ Real-time pollution monitoring")
    print("✅ AI-powered predictions")
    print("✅ Citizen reporting system")
    print("✅ Drone deployment interface")
    print("✅ Blockchain transparency")
    print("✅ Multi-city analytics")

    demo.launch(
        share=True,
        server_name="0.0.0.0",
        server_port=7860,
        show_error=True,
        debug=True
    )

Note: threading may already be installed
Note: queue may already be installed
Note: json may already be installed
Note: time may already be installed
Note: random may already be installed


/tmp/ipython-input-1-5456123.py:83: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(start='2024-01-01', end='2025-01-01', freq='H')


AI Model initialized and trained successfully!
🌍 EcoGuard AI System Starting...
Features loaded:
✅ Real-time pollution monitoring
✅ AI-powered predictions
✅ Citizen reporting system
✅ Drone deployment interface
✅ Blockchain transparency
✅ Multi-city analytics
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://82ec5c3f3c620b6448.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
